# 3.1 Managed Services

# 3.2 Cloud Functions (GCP)

## Echo Service

In [1]:
!cat scripts/gcp_cloud_function/echo/requirements.txt

flask

In [3]:
!cat scripts/gcp_cloud_function/echo/main.py

def echo(request):
    import flask

    data = {"success": False}
    params = request.get_json()

    if "msg" in params:
        data["response"] = str(params['msg'])
        data["success"] = True
    
    return flask.jsonify(data)

In [6]:
URL_ECHO_SERVICE = "https://us-central1-semiotic-method-277520.cloudfunctions.net/echo"

In [7]:
import requests

result = requests.post(
    url=URL_ECHO_SERVICE, 
    json={'msg': 'Hello from Cloud Function'}
)
print(result.json())

{'response': 'Hello from Cloud Function', 'success': True}


## Cloud Storage (GCS)

In [44]:
from google.cloud import storage
bucket_name = "dsp_model_store_00"

storage_client = storage.Client()
storage_client.create_bucket(bucket_name)

for bucket in storage_client.list_buckets():
    print(bucket.name)

dsp_model_store_00


In [1]:
# save to GCS
from google.cloud import storage

bucket_name = "dsp_model_store_00"
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

blob = bucket.blob("serverless/logit/v1")
blob.upload_from_filename("logit.pkl")

blob = bucket.blob("serverless/keras/v1")
blob.upload_from_filename("games.h5")

In [46]:
# load from GCS
import joblib
from google.cloud import storage

bucket_name = "dsp_model_store_00"
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

blob = bucket.blob("serverless/logit/v1")
blob.download_to_filename("/tmp/local_logit.pkl")

model = joblib.load("/tmp/local_logit.pkl")
model

LogisticRegression()

## Model Function

In [16]:
!cat scripts/gcp_cloud_function/pred/requirements.txt

google-cloud-storage
sklearn
pandas
flask
joblib

In [17]:
!cat scripts/gcp_cloud_function/pred/main.py

def pred(request):

    import flask
    import joblib
    import sklearn
    import pandas as pd
    from google.cloud import storage

    data = {"success": False}
    params = request.get_json()
    
    if "G1" in params:
        new_row = {"G1": params.get("G1"), "G2": params.get("G2"),
                   "G3": params.get("G3"), "G4": params.get("G4"),
                   "G5": params.get("G5"), "G6": params.get("G6"),
                   "G7": params.get("G7"), "G8": params.get("G8"),
                   "G9": params.get("G9"), "G10":params.get("G10")}
        
        new_x = pd.DataFrame.from_dict(data=new_row, 
                                       orient="index",
                                       dtype="float").T

        # set up access to the GCS bucket
        bucket_name = "dsp_model_store_00"
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(bucket_name)

        # download and load the model
        blob = bucket.blob("serverless/lo

In [18]:
URL_PRED_SERVICE = "https://us-central1-semiotic-method-277520.cloudfunctions.net/pred"

In [36]:
import requests

result = requests.post(
    url=URL_PRED_SERVICE,
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'})
print(result.json())

{'response': '0.06730006696024816', 'success': True}


In [40]:
%%timeit
result = requests.post(
    url=URL_PRED_SERVICE,
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'})

337 ms ± 28.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
!cat scripts/gcp_cloud_function/pred/main_fast.py

model = None

def pred_fast(request):
    global model

    import flask
    import joblib
    import sklearn
    import pandas as pd
    from google.cloud import storage

    data = {"success": False}
    params = request.get_json()
    
    if "G1" in params:
        new_row = {"G1": params.get("G1"), "G2": params.get("G2"),
                   "G3": params.get("G3"), "G4": params.get("G4"),
                   "G5": params.get("G5"), "G6": params.get("G6"),
                   "G7": params.get("G7"), "G8": params.get("G8"),
                   "G9": params.get("G9"), "G10":params.get("G10")}
        
        new_x = pd.DataFrame.from_dict(data=new_row, 
                                       orient="index",
                                       dtype="float").T

        if not model:
            # set up access to the GCS bucket
            bucket_name = "dsp_model_store_00"
            storage_client = storage.Client()
            bucket = storage_client.get_bucket(bucket_name)

     

In [38]:
URL_PRED_FAST_SERVICE = "https://us-central1-semiotic-method-277520.cloudfunctions.net/pred_fast"

In [39]:
import requests

result = requests.post(
    url=URL_PRED_FAST_SERVICE,
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'})
print(result.json())

{'response': '0.06730006696024816', 'success': True}


In [41]:
%%timeit
result = requests.post(
    url=URL_PRED_FAST_SERVICE,
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'})

105 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Keras Model

In [15]:
!cat scripts/gcp_cloud_function/predict/requirements.txt

google-cloud-storage
tensorflow-cpu
pandas
flask

In [16]:
!cat scripts/gcp_cloud_function/predict/main.py

model = None

def predict(request):
    global model

    import flask
    import pandas as pd
    import tensorflow.keras as K
    from google.cloud import storage

    data = {"success": False}
    params = request.get_json()
    
    if "G1" in params:
        new_row = {"G1": params.get("G1"), "G2": params.get("G2"),
                   "G3": params.get("G3"), "G4": params.get("G4"),
                   "G5": params.get("G5"), "G6": params.get("G6"),
                   "G7": params.get("G7"), "G8": params.get("G8"),
                   "G9": params.get("G9"), "G10":params.get("G10")}
        
        new_x = pd.DataFrame.from_dict(data=new_row, 
                                       orient="index",
                                       dtype="float").T

        # download model if not cached
        if not model:
            # set up access to the GCS bucket
            bucket_name = "dsp_model_store_00"
            storage_client = storage.Client()
            bucket = storage_clie

In [17]:
URL_PREDICT_SERVICE = "https://us-central1-semiotic-method-277520.cloudfunctions.net/predict"

In [18]:
import requests

result = requests.post(
    url=URL_PREDICT_SERVICE,
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'})
print(result.json())

{'response': '0.03087676', 'success': True}


# 3.3 Lambda Functions (AWS)

## Echo Function

In [1]:
!cat scripts/aws_lambda_function/echo/lambda_function.py

def lambda_handler(event, context):
    
    return {
        'statusCode': 200,
        'body': event['msg']
    }

## Simple Storage Service (S3)

In [1]:
# !aws configure
# !aws s3 ls

## Model Function

In [5]:
!cat scripts/aws_lambda_function/logit/logit.py

import json
import joblib
import pandas as pd


model = joblib.load('logit.pkl')


def lambda_handler(event, context):
    # read in the request body as the event dict
    if "body" in event:
        event = event["body"]
        event = json.loads(event) if event else {}
    
    if "G1" in event:
        new_row = {"G1": event["G1"], "G2": event["G2"],
                   "G3": event["G3"], "G4": event["G4"],
                   "G5": event["G5"], "G6": event["G6"], 
                   "G7": event["G7"], "G8": event["G8"], 
                   "G9": event["G9"], "G10": event["G10"]}
        
        new_x = pd.DataFrame.from_dict(data=new_row, 
                                       orient="index",
                                       dtype="float").T
        
        prediction = str(model.predict_proba(new_x)[0][1])
        
        return {"body": "Prediction " + prediction}
    
    return {"body": "No parameters"}

In [11]:
import requests

result = requests.post(
    url="https://c5flbent7a.execute-api.us-east-2.amazonaws.com/default/logit",
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'}
)
print(result.text)

Prediction 0.06730006696024816


## Keras Model

In [4]:
!cat scripts/aws_lambda_function/keras/keras.py

import json
import pandas as pd
import tensorflow.keras as K


model = K.models.load_model("games.h5")


def lambda_handler(event, context):
    # read in the request body as the event dict
    if "body" in event:
        event = event["body"]
        event = json.loads(event) if event else {}
    
    if "G1" in event:
        new_row = {"G1": event["G1"], "G2": event["G2"],
                   "G3": event["G3"], "G4": event["G4"],
                   "G5": event["G5"], "G6": event["G6"], 
                   "G7": event["G7"], "G8": event["G8"], 
                   "G9": event["G9"], "G10": event["G10"]}
        
        new_x = pd.DataFrame.from_dict(data=new_row, 
                                       orient="index",
                                       dtype="float").T
        
        prediction = str(model.predict(new_x)[0][0])
        
        return {"body": "Prediction " + prediction}
    
    return {"body": "No parameters"}

In [8]:
from scripts.aws_lambda_function.keras.keras import lambda_handler

lambda_handler(
    event={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
           'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'},
    context=None
)

{'body': 'Prediction 0.03087676'}

Unfortunately, <i>"unzipped size must be smaller than 262144000 bytes"</i> in order to be deployed as Lambda function